# Model management with MLflow

Model management can be done using both MLflow and Azure ML SDK/CLI v2. If you are familiar with MLflow and the capabilities it exposes, we support the entire model lifecycle using the MLFlow client. If you rather use Azure ML specific features or do model management using the CLI, in the same way you can manage the lifecycle using the Azure ML CLI/SDK v2.

## Support matrix for managing models

The MLflow client exposes several methods to retrieve and manage models. The following table shows which of those methods are currently supported in MLflow when connected to Azure ML:

| Feature | MLflow | Azure ML with MLflow | Azure ML CLIv2 |
| :- | :-: | :-: | :-: |
| Registering models in MLModel models | ☑️ | ☑️ | ☑️ |
| Registering models not in MLModel format | ☐ | ☐ | ☑️ |
| Registering models from runs with URIs as `runs/:<ruin-id>/<path>` | ☑️ | ☑️ | ☐ |
| Registering models from runs with URIs as `azureml://jobs/<job-id>/outputs/artifacts/<path>` | ☐ | ☐ | ☑️* |
| Listing registered models | ☑️ | ☑️ | ☑️ |
| Retrieving details of registered model's versions | ☑️ | ☑️ | ☑️ |
| Editing registered model's versions description | ☑️ | ☑️ | ☑️ |
| Editing registered model's versions tags | ☑️ | ☑️ | ☑️ |
| Renaming registered models | ☑️ | ☐** | ☐** |
| Deleting a registered model (container) | ☑️ | ☐** | ☐** |
| Deleting a registered model's version | ☑️ | ☑️ | ☑️ |
| Search registered models by name | ☑️ | ☑️ | ☑️ |
| Search registered models using string comparators `LIKE` and `ILIKE` | ☑️ | ☐ | ☐ |
| Search registered models by tag | ☐ | ☐ | ☐ |

Notes:
* (*) With compatibility issues with Mlflow client
* (**) Registered models are immutable objects in Azure ML

## Prerequisites to run this notebook

In [ ]:
# Ensure you have the dependencies for this notebook
%pip install -r logging_model_with_mlflow.txt

In [ ]:
import mlflow

In the following notebook, we will explore an example that uses the following naming convention:

In [ ]:
experiment_name = "heart-classifier"
model_name = "heart-classifier"
artifact_path = "classifier"

We need to create a couple of runs and experiments in the workspace to work with. Please run at least one or two training routines:

In [ ]:
# Install the AML extension and log into Azure
!az extension add -n ml
!az login

In [ ]:
# Configure workspace and resource group
!az config set defaults.workspace=MyWorkspace defaults.group=MyResourceGroup

In [ ]:
# Ensure there is a compute to train on
!az ml compute create -f ../jobs/trainer-cpu.compute.yml

In [ ]:
# Submit a couple of training jobs to have something to work with
!az ml job create -f ../jobs/heart-classifier.job.yml

## Before starting

If you are running inside of a Compute Instance in Azure ML, MLflow is already configured to be used. If you are running in you local machine or in a different platform, please configure MLflow to point to the workspace you want to work with by uncommenting the following line and placing your workspace tracking URL.

In [ ]:
# mlflow.set_tracking_uri = "<TRACKING_URI>"

> To get the URI, please navigate to Azure ML Studio and select the workspace you are working on > Click on the name of the workspace at the upper right corner of the page > Click “View all properties in Azure Portal” on the pane popup > Copy the MLflow tracking URI value from the properties section.

## Creating models from an existing run

If you have an Mlflow model logged inside of a run and you want to register it in a registry, you can do that by using the experiment and run ID information from the run:

In [ ]:
exp = mlflow.get_experiment_by_name(experiment_name)
last_run = mlflow.search_runs(exp.experiment_id, output_format="list")[-1]
print(last_run.info.run_id)

Once we have the run identified, we can register the model using Mlflow client:

In [ ]:
mlflow.register_model(f"runs:/{last_run.info.run_id}/{artifact_path}", model_name)

## Creating models from assets

If you have a folder with an MLModel MLflow model, then you can register it directly. There is no need for the model to be always in the context of a run. To do that you can use the URI schema `file://path/to/model` to register it. Let's create a simple model and save it in MLModel format:

In [ ]:
from sklearn import linear_model

reg = linear_model.LinearRegression()
reg.fit([[0, 0], [1, 1], [2, 2]], [0, 1, 2])

mlflow.sklearn.save_model(reg, "./regressor")

Check the files in the folder

In [ ]:
!ls regressor

You can now register the model from the local path:

In [ ]:
import os

model_local_path = os.path.abspath("./regressor")

In [ ]:
mlflow.register_model(f"file://{model_local_path}", "local-model-test")

> Notice how the model URI schema `file:/` requires absolute paths.

## Querying models

### Querying all the models in the registry

You can query all the registered models in the registry using the MLflow client with the method `list_registered_models`.

In [ ]:
client = mlflow.tracking.MlflowClient()

In [ ]:
for model in client.list_registered_models():
    print(f"{model.name}")

If you are not sure of the name of the model you are looking for, you can search for it:

In [ ]:
client.search_registered_models(f"name='{model_name}'")

### Getting specific versions of the model

The command above will retrieve the model object which contains all the model versions. However, if you want to get the last registered model version of a given model, you can use `get_registered_model`:

In [ ]:
client.get_registered_model(model_name)

If you need an specific version of the model, you can indicate so:

In [ ]:
client.get_model_version(model_name, version=2)

## Model stages

MLflow supports model's stages to manage model's lifecycle. Stage are assigned to model's version (instead of models). This means that a given model can have multiple versions on different stages.

### Queying model stages

You can use the MLflow client to check all the possible stages a model can be:

In [ ]:
client.get_model_version_stages(model_name, version="latest")

You can see what model version is on each stage by getting the model from the registry:

In [ ]:
client.get_latest_versions(model_name, stages=["Staging"])

Notice that multiple versions can be in the same stage at the same time in Mlflow, however, this method returns the latest version (greater version) among all of them.

> Caution: Notice that stages are case sensitive.

### Transitioning models

To transition a model to a particular stage, you can:

In [ ]:
client.transition_model_version_stage(model_name, version=3, stage="Staging")

By default, if there were an existing model version in that particular stage, it will remain there. Hence, it won't be replaced. Alternatively, you can indicate `archive_existing_versions=True` to tell MLflow to move the existing model's version to the stage `Archived`.

In [ ]:
client.transition_model_version_stage(
    model_name, version=3, stage="Staging", archive_existing_versions=True
)

### Loading models from stages

You can load a model in a particular stage directly from Python using the `load_model` function and the following `URI` format. Notice that for this method to success, you need to have all the libraries and dependencies already installed in the environment you are working at.

In [ ]:
model = mlflow.pyfunc.load_model(f"models:/{model_name}/Staging")

## Editing and deleting models

Editing registered models is supported in both Mlflow and Azure ML, however, there are some differences between them that are important to notice:

### Editing models

You can edit model's description and tags from a model using Mlflow:

> Renaming models is not supported in Azure ML as model objects are immmutable.

In [ ]:
client.update_model_version(
    model_name, version=1, description="A heart condition classifier"
)

To edit tags, you have to use the method `set_model_version_tag` and `remove_model_version_tag`:

In [ ]:
client.set_model_version_tag(
    model_name, version="1", key="type", value="classification"
)

Removing a tag:

In [ ]:
client.delete_model_version_tag(model_name, version="1", key="type")

### Deleting a model version

You can delete any model version in the registry using the MLflow client. However, Azure ML doesn't support deleting the entire model container. To achieve the same thing, you will need to delete all the model versions from a given model.

In [ ]:
import mlflow

In [ ]:
client.delete_model_version(model_name, version="2")